In [1]:
import os
import sys
from scipy.integrate import simps
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy as np
import json
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
from scipy.stats import spearmanr


project_dir = "D:/MDSI_project/"
func_dir = "ANSYS_Building_model/PyAPDL_example/FUNC"
doe_dir = "ANSYS_Building_model/PyAPDL_example/DOE_data"
sys.path.append(os.path.join(project_dir, func_dir))
sys.path.append(os.path.join(project_dir, doe_dir))

func_dir_ = os.path.join(project_dir, func_dir)
doe_dir_ = os.path.join(project_dir, doe_dir)

from Simulation_PyAPDL import simulation_PyAPDL
import func_FEMU as FEMU

In [2]:
simu_tk = simulation_PyAPDL(nproc=4,nerr=10000)
simu_tk.launch_engine()
simu_tk.mapdl.clear()

PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Launch Pymapdl
     Launch Pymapdl successfully, duration 19.3191 seconds.


- fix parameter: all of the nu
- vary parameter: t, e, rho, vs, dr, height
- Assumption: x_slab = t_stair
- Number of parameter reduced to: 19

### FEM modal analysis

In [42]:
import os
from IPython.display import clear_output

#x = [0.84507014, 0.13539517760383665, 1.0, 0.50633148, 0.1885995, 0.75976072, 1.0, 1.0, 0.21309783, 0.5004223475169194, 0.66487426, 0.7993062197679878, 0.6071382305792775, 0.23569906134107818, 0.9707206995435914, 0.35020139, 0.35176575, 0.54754628, 0.0]

#x =  [0.84507014, 0.10531804576144854        ,-0.2477233686953787        ,0.50633148 ,0.1885995  ,0.75976072,
#                1.35       , 1.35        ,0.21309783 ,0.5042891338846919        ,0.66487426 ,-0.11111751970865216       ,
#                -0.35       , -0.1706996362423985        ,1.35        ,0.35020139 ,0.35176575 ,0.54754628,
#                0.11599825721116747       ]

x =  [0.84507014, 0.5        ,0.5        ,0.50633148 ,0.1885995  ,0.75976072,
                0.5       , 0.5        ,0.21309783 ,0.5        ,0.66487426 ,0.5       ,
                0.5       , 0.5        ,0.5        ,0.35020139 ,0.35176575 ,0.54754628,
                0.5       ]
DOE_info = {}
vary_ratio = 0.5

# Transform X from [0,1] to [-1,1]
X_transformed = 2 * np.array(x) - 1
X_transformed = X_transformed.reshape(-1,1)
X_transformed = np.transpose(X_transformed)
i_doe = 0
X_doe_transformed = X_transformed

    
simu_tk.mapdl.clear()

# Parameter list    = [slab ,corridor, ground, edge_wall, inner_wall, stair ]
# Parameter list    = [ vs,   rho,   nu ]
t_para   = np.array([ 0.468+ vary_ratio*(0.468)*X_transformed[i_doe,0],
                      0.8  + vary_ratio*(0.8  )*X_transformed[i_doe,1],
                      0.05 + vary_ratio*(0.05 )*X_transformed[i_doe,2],
                      0.2  + vary_ratio*(0.2  )*X_transformed[i_doe,3],
                      0.2  + vary_ratio*(0.2  )*X_transformed[i_doe,4],
                      0.468+ vary_ratio*(0.468)*X_transformed[i_doe,0]])
#
e_para   = np.array([ 20e9 + vary_ratio*20e9*X_transformed[i_doe,5],
                      20e9 + vary_ratio*20e9*X_transformed[i_doe,6],
                      20e9 + vary_ratio*20e9*X_transformed[i_doe,7],
                      20e9 + vary_ratio*20e9*X_transformed[i_doe,8],
                      20e9 + vary_ratio*20e9*X_transformed[i_doe,9],
                      20e9 + vary_ratio*20e9*X_transformed[i_doe,5]])
#
nu_para  = np.array([ 0.27,
                      0.27,
                      0.27,
                      0.27,
                      0.27,
                      0.27])
#
rho_para = np.array([ 2300 + vary_ratio*2300*X_transformed[i_doe,10],
                      2300 + vary_ratio*2300*X_transformed[i_doe,11],
                      2300 + vary_ratio*2300*X_transformed[i_doe,12],
                      2300 + vary_ratio*2300*X_transformed[i_doe,13],
                      2300 + vary_ratio*2300*X_transformed[i_doe,14],
                      2300 + vary_ratio*2300*X_transformed[i_doe,10]])
#
soil_para = np.array([ 350  + vary_ratio*(350 )*X_transformed[i_doe,15],
                       1400 + vary_ratio*(1400)*X_transformed[i_doe,16],
                       0.33 ])
#
height_b            = 3.37 + vary_ratio*3.37*X_transformed[i_doe,17]
damp_ratio_b        = 0.045+ 0.01*X_transformed[i_doe,18]
#
simu_tk.setting_parameter(bool_SSI= True,height=height_b, t_arr=t_para, e_arr=e_para, nu_arr=nu_para, rho_arr=rho_para, soil_arr=soil_para)
simu_tk.build_model()
#
#
# Excitation point: Hammer 2.0 Pos1 (shaker)
tol = 0.3
simu_tk.mapdl.nsel('S','LOC','Z',simu_tk.height*2)
simu_tk.mapdl.nsel('R','LOC','X',9.3-tol,9.3+tol)
nsel_id_1 = simu_tk.mapdl.nsel('R','LOC','Y',5.9-tol,5.9+tol)
simu_tk.mapdl.nsel('ALL')
#
# Apply the force on selected node and solve 
simu_tk.solve_model(Solu_type=2, N_modes = 40, Freq_Incr=0.5, End_Freq=50,Damping_ratio= damp_ratio_b, excitation_node=nsel_id_1[1])
#
#
modal_info = simu_tk.get_all_modal_info(N_modes_exp=40)
#
#


result =simu_tk.mapdl.result



 Setting information
     Building inner wall           :  True
     Building stair                :  True
     Building SSI                  :  True
     Element size                  :  0.4
     Height of building            :  3.5302309636
 Assigning parameter
     Rewrite thickness by input
     Rewrite Young's modulus by input
     Rewrite Poisson's ratio by input
     Rewrite Density by input
     Rewrite soil properties by input
 /PREP7 activated
     /PREP7 finished, duration 7.5767 seconds.
Solve model: /SOLU
 Setting information
     Solution type                 :  2
     Start frequency               :  0.001
     End frequency                 :  50
     Freqency increment            :  0.5
     Number of modes (solu_type=2) :  40
     Damping ratio                 :  0.045
     /SOLU finished, duration 5.6830 seconds.
 Extract all modal information , duration 4.9941 seconds.


In [43]:
import pyvista as pv
pv.global_theme.background = 'white'

In [49]:
# 3, 9 

result.plot_nodal_displacement(10,comp = 'Z', show_displacement=True, displacement_factor=500)

Widget(value='<iframe src="http://localhost:51911/index.html?ui=P_0x268ca6303a0_40&reconnect=auto" class="pyvi…

In [53]:
result.plot_nodal_displacement(3,comp = 'Z', show_displacement=True, displacement_factor=500)


Widget(value='<iframe src="http://localhost:51911/index.html?ui=P_0x268e2b78cd0_44&reconnect=auto" class="pyvi…

In [33]:
simu_tk.mapdl.post1()
simu_tk.mapdl.set(1, 10)
#simu_tk.mapdl.upcoord(1)
simu_tk.mapdl.post_processing.plot_nodal_displacement('Z',show_edges=True,smooth_shading=False)

Widget(value='<iframe src="http://localhost:51911/index.html?ui=P_0x268784dda50_27&reconnect=auto" class="pyvi…

In [55]:
simu_tk.mapdl.aplot(show_area_numbering=False, color_areas=True,)

Widget(value='<iframe src="http://localhost:51911/index.html?ui=P_0x269386db9a0_46&reconnect=auto" class="pyvi…